<h3 style="color: #7aa2f7; font-weight: bold;" align="center">Model Training</h3>

In [1]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.207 🚀 Python-3.10.13 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Setup complete ✅ (16 CPUs, 7.4 GB RAM, 97.0/1006.9 GB disk)


In [2]:
from ultralytics import YOLO

In [3]:
# Transfer Learning: Fine Tunning
# Load the pre-trained YOLOv8 model
# model = YOLO(model="../models/yolov8/yolov8n.pt")
# model = YOLO(model="../models/yolov8/yolov8m.pt")
model = YOLO(model="yolov8m.pt")

### Model Info


In [4]:
model.info()

YOLOv8m summary: 295 layers, 25902640 parameters, 0 gradients, 79.3 GFLOPs


(295, 25902640, 0, 79.3204224)

In [5]:
model.model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
   

In [6]:
classes = []
components = []

count = 1
for index, info in enumerate(model.named_parameters()):
    # info[0]: name
    # info[1]: params
    if index == 0:
        print(f"name: {info[0]} \nparams: {info[1][0]}")

    if count == 1 and info[0].split(".")[2] == "conv":
        count = 0
        print(info[1].required_grade)

    classes.append(info[0].split(".")[2])
    components.append(info[0].split(".")[3])

set(classes), set(components)

name: model.model.0.conv.weight 
params: tensor([[[ 0.1273,  0.1281, -0.0081],
         [-0.1766, -0.1777, -0.0365],
         [ 0.0451,  0.0630,  0.0426]],

        [[ 0.1752,  0.1748,  0.0518],
         [-0.2291, -0.2468, -0.0364],
         [ 0.0392,  0.0364,  0.0303]],

        [[ 0.0283,  0.0414, -0.0014],
         [-0.0787, -0.0924, -0.0219],
         [ 0.0566,  0.0406,  0.0180]]])


({'0',
  '1',
  '12',
  '15',
  '16',
  '18',
  '19',
  '2',
  '21',
  '22',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9'},
 {'bn', 'conv', 'cv1', 'cv2', 'cv3', 'dfl', 'm'})

### Model Freeze


In [7]:
layer_names = []
for index, info in enumerate(model.named_parameters()):
    # info[0]: name
    # info[1]: params
    layer_names.append(info[0])

In [8]:
# Layers to be freeze
count_last_layers_freeze = 10
to_freeze_layer_names = layer_names[
    -count_last_layers_freeze - 1 : -1
]  # -1: don't freeze output layer `model.model.22.dfl.conv.weight`

to_freeze_layer_names

['model.model.22.cv3.1.2.weight',
 'model.model.22.cv3.1.2.bias',
 'model.model.22.cv3.2.0.conv.weight',
 'model.model.22.cv3.2.0.bn.weight',
 'model.model.22.cv3.2.0.bn.bias',
 'model.model.22.cv3.2.1.conv.weight',
 'model.model.22.cv3.2.1.bn.weight',
 'model.model.22.cv3.2.1.bn.bias',
 'model.model.22.cv3.2.2.weight',
 'model.model.22.cv3.2.2.bias']

In [9]:
# Freezing the layer
for index, info in enumerate(model.named_parameters()):
    # info[0]: name
    # info[1]: params

    info[1].requires_grad = True  # train all layers
    for name in to_freeze_layer_names:
        if info[0] == name:
            info[1].requires_grad = False  # Freeze the layers

In [10]:
def freeze_layers_fn(trainer):
    model = trainer.model
    layer_names = []

    for index, info in enumerate(model.named_parameters()):
        # info[0]: name
        # info[1]: params
        layer_names.append(info[0])

    # Layers to be freeze
    count_last_layers_freeze = 10
    to_freeze_layer_names = layer_names[
        -count_last_layers_freeze - 1 : -1
    ]  # -1: don't freeze output layer `model.model.22.dfl.conv.weight`

    for index, info in enumerate(model.named_parameters()):
        # info[0]: name
        # info[1]: params

        info[1].requires_grad = True  # train all layers
        for name in to_freeze_layer_names:
            if info[0] == name:
                info[1].requires_grad = False  # Freeze the layers

In [11]:
import torch

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [13]:
# Replace the final classification layer of the pre-trained YOLOv8 model with a new,
# randomly initialized classification layer that has the number of
# output classes equal to the number of classes in your new dataset

# model.nc = 2
# model.add_callback("on_train_start", freeze_layers_fn)  # callback for freeze layers

In [14]:
# Use the model
model.train(
    data="../datasets/config.yaml",
    task="detect",
    batch=16,
    device=device,
    imgsz=640,
    pretrained=True,
    rect=False,
    freeze=to_freeze_layer_names,
    val=True,
    # save_dir="results/detect/train", # not working
    seed=7,
    epochs=15,
    resume=False # By setting resume=True, the train function will continue training from where it left off, using the state stored in the 'path/to/last.pt' file. If the resume argument is omitted or set to False, the train function will start a new training session.
)  # train the model

engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=../datasets/config.yaml, epochs=15, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=7, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=['model.model.22.cv3.1.2.weight', 'model.model.22.cv3.1.2.bias', 'model.model.22.cv3.2.0.conv.weight', 'model.model.22.cv3.2.0.bn.weight', 'model.model.22.cv3.2.0.bn.bias', 'model.model.22.cv3.2.1.conv.weight', 'model.model.22.cv3.2.1.bn.weight', 'model.model.22.cv3.2.1.bn.bias', 'model.model.22.cv3.2.2.weight', 'model.model.22.cv3.2.2.bias'], overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_c

In [ ]:
# Predict with the model

test_imgs = "../datasets/farm_animals/test"
results_detection = model.predict(
    source=test_imgs,
    task="detect",
    model="runs/detect/train/weights/best.pt",
)

In [ ]:
# boxes coordinate
# results_detection[0].boxes.xyxy
no_test_img = 10
test_img_data_boxes = []
for i in range(no_test_img):
  test_img_data_boxes.append(results_detection[i].boxes)

In [ ]:
for i in test_img_data_boxes:
  print(f"\nCLASS: {i.cls}\n-----")
  print(i)
  print("----------------------------------------------")

In [ ]:
path = model.export(format="onnx")